### Running Chat without Preloading the Model

In [3]:
import ollama
import httpx

In [2]:
url = 'https://example.com/api'
print(f"Connecting to URL: {url}")

Connecting to URL: https://example.com/api


In [4]:
try:
    response = ollama.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': 'What is 1 + 1?',
        },
    ])
    print(response)
except httpx.ConnectError as e:
    print(f"Connection error: {e}")

{'model': 'llama3', 'created_at': '2024-07-27T15:36:07.5562961Z', 'message': {'role': 'assistant', 'content': "That's an easy one!\n\nThe answer to 1 + 1 is... (drumroll please)... 2!"}, 'done_reason': 'stop', 'done': True, 'total_duration': 22916681600, 'load_duration': 14320482800, 'prompt_eval_count': 18, 'prompt_eval_duration': 2017832000, 'eval_count': 26, 'eval_duration': 6572829000}


In [3]:
try:
    response = ollama.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': 'Why is the sky blue?',
        },
    ])
    print(response)
except httpx.ConnectError as e:
    print(f"Connection error: {e}")

{'model': 'llama3', 'created_at': '2024-07-27T14:06:37.2676516Z', 'message': {'role': 'assistant', 'content': "The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. **Light from the sun**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).\n2. **Scattering by tiny molecules**: As this light travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.\n3. **Blue light is scattered more**: When the light hits these tiny molecules, some of the shorter wavelengths, like blue and violet, are scattered in all directions by the molecules. This scattering effect is more pronounced for shorter wavelengths because they have a higher frequency and are

In [4]:
response

{'model': 'llama3',
 'created_at': '2024-07-27T14:06:37.2676516Z',
 'message': {'role': 'assistant',
  'content': "The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. **Light from the sun**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).\n2. **Scattering by tiny molecules**: As this light travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.\n3. **Blue light is scattered more**: When the light hits these tiny molecules, some of the shorter wavelengths, like blue and violet, are scattered in all directions by the molecules. This scattering effect is more pronounced for shorter wavelengths because they have a higher frequency and

### Preloaded Model

In [1]:
import requests
import json

# Define the API endpoint and model details
api_url = "http://localhost:11434/api/chat"
model_name = "llama3"

# Function to preload the model
def preload_model(api_url, model_name):
    headers = {'Content-Type': 'application/json'}
    data = {'model': model_name}
    response = requests.post(api_url, headers=headers, data=json.dumps(data))
    
    if response.status_code == 200:
        print(f"Model {model_name} preloaded successfully.")
    else:
        print(f"Failed to preload model {model_name}. Response code: {response.status_code}")
        print(response.text)

# Preload the model
preload_model(api_url, model_name)

Model llama3 preloaded successfully.


In [4]:
# Function to generate text using the preloaded model
def generate_text(generate_url, model_name, prompt):
    headers = {'Content-Type': 'application/json'}
    data = {
        'model': model_name,
        'messages': [
            {
                'role': 'user',
                'content': prompt,
            }
        ],
        'stream': True
    }

    try: 
        response = requests.post(generate_url, headers=headers, data=json.dumps(data), stream=True)

        complete_message = []
        
        # Iterate over the streamed response
        for chunk in response.iter_lines():
            if chunk:
                try:
                    # Decode the JSON chunk
                    json_chunk = json.loads(chunk)
                    
                    # Append content to the message list
                    content = json_chunk.get('message', {}).get('content', '')
                    if content:
                        complete_message.append(content)
                    
                    # Break if the response is complete
                    if json_chunk.get('done'):
                        break
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e}")
        
        # Combine all chunks into a single string
        one_string = ''.join(complete_message)
        print(one_string)

    except requests.RequestException as e:
        print(f"Request error: {e}")
    

In [5]:
generate_url = "http://localhost:11434/api/chat"
result = generate_text(generate_url, model_name, "What is 1 + 1?")
print(result)


The answer to 1 + 1 is 2.
None


### Explorigng with Generate

In [3]:
from ollama import generate

response = generate('llama3', 'what is 1 + 1?')
print(response['response'])

The answer to 1 + 1 is... 2!


In [4]:
response

{'model': 'llama3',
 'created_at': '2024-07-27T15:48:44.5627373Z',
 'response': 'The answer to 1 + 1 is... 2!',
 'done': True,
 'done_reason': 'stop',
 'context': [128006,
  882,
  128007,
  271,
  12840,
  374,
  220,
  16,
  489,
  220,
  16,
  30,
  128009,
  128006,
  78191,
  128007,
  271,
  791,
  4320,
  311,
  220,
  16,
  489,
  220,
  16,
  374,
  1131,
  220,
  17,
  0],
 'total_duration': 126820022300,
 'load_duration': 112882956700,
 'prompt_eval_count': 18,
 'prompt_eval_duration': 5682312000,
 'eval_count': 14,
 'eval_duration': 8241180000}